<div class="alert alert-info">
Ссылка для просмотра ноутбука в интерактивном режиме для использования гиперссылок и корректного отображения разметки:<br>
<a href='https://nbviewer.org/github/yulianikola/portfolio/blob/master/data_analysis_python/3_expl_data_analysis/horse_surge.ipynb'>horse_surge</a></div>

### Подготовка данных
#### Данные о состоянии здоровья лошадей с кишечными коликами

Описание данных:<br>
https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names<br>
Данные:<br>
https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv<br>

#### Задачи:
1. Изучить набор данных на основе описания его столбцов и выбрать столбцы для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них.
1. В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.
1. Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

<p id="0">
<h4>Содержание</h4>
<ul type="square"><a href="#1"><li>Знакомство с данными</li></a>
<a href="#2"><li>Дубликаты</li></a>
<a href="#3"><li>Анализ признаков (распределение, пропуски, выбросы)</li></a>

In [1]:
import pandas as pd
import re

<p id="1">
<h4>Знакомство с данными</h4><br>

Извлечем названия переменных из описания для наименования столбцов:

In [2]:
f = open("C:\\python\\portfolio\\data_analysis_python\\3_expl_data_analysis\\data\\horse_descr.txt", 'r', encoding ='utf-8')
descr = f.readlines()

In [3]:
for line in descr:
    if 'Attribute Information' in line:
        ind = descr.index(line)
        break

С помощью регулярных выражений создадим список для названий столбцов на основе описания показателей:

In [4]:
names = ''.join(descr[ind:])
names_1 = re.findall(r'(?<=\d:)\s*(?:\w+[ ]*)+',names)
names_final = list(map(lambda name: name.strip().lower().replace(' ','_'), names_1))
# Добавить два столбца, тк в описании три под одним именем: 25, 26, 27: type of lesion.
names_final.insert(names_final.index('type_of_lesion') + 1,'type_of_lesion_0')
names_final.insert(names_final.index('type_of_lesion_0') + 1,'type_of_lesion_1')
print(len(names_final))
print(names_final)

28
['surgery', 'age', 'hospital_number', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'peripheral_pulse', 'mucous_membranes', 'capillary_refill_time', 'pain', 'peristalsis', 'abdominal_distension', 'nasogastric_tube', 'nasogastric_reflux', 'nasogastric_reflux_ph', 'rectal_examination', 'abdomen', 'packed_cell_volume', 'total_protein', 'abdominocentesis_appearance', 'abdomcentesis_total_protein', 'outcome', 'surgical_lesion', 'type_of_lesion', 'type_of_lesion_0', 'type_of_lesion_1', 'cp_data']


Загружаем данные в датафрейм и присваиваем названия столбцам:

In [5]:
horse = pd.read_csv('C:\\python\\portfolio\\data_analysis_python\\3_expl_data_analysis\\data\\horse_surge.csv',
                    names = names_final)
horse.head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,...,packed_cell_volume,total_protein,abdominocentesis_appearance,abdomcentesis_total_protein,outcome,surgical_lesion,type_of_lesion,type_of_lesion_0,type_of_lesion_1,cp_data
0,2,1,530101,38.50,66,28,3,3,?,2,...,45.00,8.40,?,?,2,2,11300,0,0,2
1,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
2,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
3,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
4,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2


В датасете 300 записей, пропусков нет. Судя по предварительному просмотру выше у всех показателей числовые значения, но у большей части столбцов определился тип данных object. Это значит, что среди значений есть нечисловые. И действительно, в предварительном просмотре видны значения "?", которые видимо означают пропуски:

In [6]:
horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   surgery                      300 non-null    object
 1   age                          300 non-null    int64 
 2   hospital_number              300 non-null    int64 
 3   rectal_temperature           300 non-null    object
 4   pulse                        300 non-null    object
 5   respiratory_rate             300 non-null    object
 6   temperature_of_extremities   300 non-null    object
 7   peripheral_pulse             300 non-null    object
 8   mucous_membranes             300 non-null    object
 9   capillary_refill_time        300 non-null    object
 10  pain                         300 non-null    object
 11  peristalsis                  300 non-null    object
 12  abdominal_distension         300 non-null    object
 13  nasogastric_tube             300 no

Увеличиваем число отображаемых столбцов в pandas. Перезагружаем данные, приравнивая вопросительные знаки к пропускам:

In [7]:
pd.set_option('display.max_columns', 30)
horse = pd.read_csv('C:\\python\\portfolio\\data_analysis_python\\3_expl_data_analysis\\data\\horse_surge.csv',
                    names = names_final, na_values = '?')
horse.head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,peripheral_pulse,mucous_membranes,capillary_refill_time,pain,peristalsis,abdominal_distension,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_examination,abdomen,packed_cell_volume,total_protein,abdominocentesis_appearance,abdomcentesis_total_protein,outcome,surgical_lesion,type_of_lesion,type_of_lesion_0,type_of_lesion_1,cp_data
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,5.0,4.0,4.0,NaN,NaN,NaN,3.0,5.0,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,3.0,4.0,2.0,NaN,NaN,NaN,4.0,2.0,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,3.0,3.0,1.0,NaN,NaN,NaN,1.0,1.0,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,2.0,4.0,4.0,1.0,2.0,5.0,3.0,NaN,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2


Теперь по всем столбцам определились числовые типы данных и видно, что наличие пропусков между столбцами значительно различается:

In [8]:
horse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   surgery                      299 non-null    float64
 1   age                          300 non-null    int64  
 2   hospital_number              300 non-null    int64  
 3   rectal_temperature           240 non-null    float64
 4   pulse                        276 non-null    float64
 5   respiratory_rate             242 non-null    float64
 6   temperature_of_extremities   244 non-null    float64
 7   peripheral_pulse             231 non-null    float64
 8   mucous_membranes             253 non-null    float64
 9   capillary_refill_time        268 non-null    float64
 10  pain                         245 non-null    float64
 11  peristalsis                  256 non-null    float64
 12  abdominal_distension         244 non-null    float64
 13  nasogastric_tube    

Посчитаем долю пропусков для каждого признака:

In [9]:
for col in horse.columns:
    share_missing = horse[col].isna().mean()
    print(f'{col} - {share_missing :.1%}')

surgery - 0.3%
age - 0.0%
hospital_number - 0.0%
rectal_temperature - 20.0%
pulse - 8.0%
respiratory_rate - 19.3%
temperature_of_extremities - 18.7%
peripheral_pulse - 23.0%
mucous_membranes - 15.7%
capillary_refill_time - 10.7%
pain - 18.3%
peristalsis - 14.7%
abdominal_distension - 18.7%
nasogastric_tube - 34.7%
nasogastric_reflux - 35.3%
nasogastric_reflux_ph - 82.3%
rectal_examination - 34.0%
abdomen - 39.3%
packed_cell_volume - 9.7%
total_protein - 11.0%
abdominocentesis_appearance - 55.0%
abdomcentesis_total_protein - 66.0%
outcome - 0.3%
surgical_lesion - 0.0%
type_of_lesion - 0.0%
type_of_lesion_0 - 0.0%
type_of_lesion_1 - 0.0%
cp_data - 0.0%


Отбираем признаки для дальнейшего изучения (тип показателей на основе описания показателей в источнике):

Берем признаки:<br>
* surgery: как важный исход лечения, категориальный
* age: как важный фактор для лечения, количественный
* hospital_number: для идентификации лошадей, но не записей (лошадь могла лечиться более одного раза), категориальный
* количественные факторы, на взгляд непрофессионала важные и разносторонне характеризующие состояние:<br>
rectal_temperature, pulse, respiratory_rate, total_protein, abdomcentesis_total_protein<br>
* категориальные факторы, на взгляд непрофессионала важные и разносторонне характеризующие состояние:<br>
temperature_of_extremities, pain, abdominal_distension (отмечен как важный в описании)<br>
* outcome: как важный исход лечения, категориальный<br>
<br>
Не берем признаки:<br>
* peripheral_pulse: категориальный признак в отличие от количественного pulse, вероятно они связаны, предпочитаю количественный pulse
* abdominocentesis_appearance и nasogastric_reflux_ph: больше половины пропусков
* cp_data: отмечено как не имеющее смысла для данного датасета
* факторы, на взгляд непрофессионала неважные или "дублирующие" по смыслу уже отобранные:<br>
surgical_lesion, mucous_membranes, capillary_refill_time, peristalsis, nasogastric_tube, nasogastric_reflux,<br>
rectal_examination, abdomen, packed_cell_volume<br>
* type_of_lesion, type_of_lesion_0, type_of_lesion_1: система типологии случаев, есть нулевые значения, по сути пропуски

Датафрейм с отобранными столбцами:

In [10]:
horse_sample = (horse.loc[:,['surgery','age','hospital_number',
                       'rectal_temperature','pulse','respiratory_rate',
                       'temperature_of_extremities','abdominal_distension',
                       'pain','outcome','abdomcentesis_total_protein','total_protein']])

<a href="#0"><h4>Наверх</h4></a>

<p id="2">
<h4>Дубликаты</h4><br>

Проверим наличие дубликатов по hospital_number, как и ожидалось из описания датасета такие есть, номера дублируются в 16 записях:

In [11]:
len(horse_sample) - horse_sample.hospital_number.nunique()

16

Проверим наличие дубликатов полных строк, таких 22 записи:

In [12]:
horse_sample.duplicated(keep = False).value_counts()

False    278
True      22
dtype: int64

Посмотрим на эти строки, значения в столбцах частично заполнены, частично есть пропуски:

In [13]:
horse_sample[horse_sample.duplicated(keep = False)]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
26,1.0,1,530526,38.3,72.0,30.0,4.0,3.0,3.0,1.0,3.9,7.0
34,1.0,1,529796,NaN,100.0,30.0,3.0,4.0,5.0,1.0,NaN,6.6
48,1.0,1,528890,38.9,80.0,44.0,3.0,3.0,2.0,2.0,NaN,6.5
59,2.0,1,528904,NaN,96.0,NaN,3.0,4.0,5.0,2.0,NaN,NaN
71,1.0,1,529424,37.8,48.0,28.0,1.0,1.0,1.0,1.0,7.0,5.9
73,1.0,1,528469,38.1,52.0,24.0,1.0,1.0,4.0,2.0,2.6,7.5
78,1.0,1,530693,NaN,NaN,NaN,3.0,3.0,5.0,2.0,NaN,5.9
83,1.0,1,5279822,38.0,NaN,24.0,3.0,4.0,5.0,2.0,NaN,7.8
94,2.0,1,528729,37.5,48.0,30.0,4.0,1.0,NaN,1.0,NaN,8.6
110,1.0,1,527544,NaN,70.0,16.0,3.0,2.0,2.0,2.0,NaN,7.5


Про дубликаты строк в описании ничего не сказано, могут ли они существовать в датасете без ошибки? Можно было бы предположить, что причина дубликатов это появление лошади в клинике в течение дня, например, и поэтому показатели одинаковые. Сгруппируем дубликаты по номеру и исходу. Среди дубликатов есть лошади с исходом смерти и усыплением, с точки зрения здравого смысла такого быть не может:

In [14]:
(pd.DataFrame(horse_sample[horse_sample.duplicated(keep = False)].
              groupby(['hospital_number','outcome'])['surgery'].count()).
 sort_index(level = [1]))

,,surgery
hospital_number,outcome,
528729,1.0,2
529424,1.0,2
529796,1.0,2
530526,1.0,2
527544,2.0,2
528469,2.0,2
528890,2.0,2
528904,2.0,2
530693,2.0,2


Удалим лишние дублирущиеся строки:

In [15]:
horse_sample.drop_duplicates(keep = 'first', inplace = True)

После этого все равно остались дубли по номерам, но они, видимо, как раз относятся к случаям нескольких лечений одной лошади:

In [16]:
len(horse_sample) - horse_sample.hospital_number.nunique()

5

Число строк сократилось с 300 до 289:

In [17]:
horse_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 0 to 299
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   surgery                      288 non-null    float64
 1   age                          289 non-null    int64  
 2   hospital_number              289 non-null    int64  
 3   rectal_temperature           233 non-null    float64
 4   pulse                        267 non-null    float64
 5   respiratory_rate             234 non-null    float64
 6   temperature_of_extremities   233 non-null    float64
 7   abdominal_distension         233 non-null    float64
 8   pain                         235 non-null    float64
 9   outcome                      288 non-null    float64
 10  abdomcentesis_total_protein  99 non-null     float64
 11  total_protein                258 non-null    float64
dtypes: float64(10), int64(2)
memory usage: 29.4 KB


<a href="#0"><h4>Наверх</h4></a>

<p id="3">
<h4>Анализ признаков: распределение, выбросы, пропуски</h4><br>
Изучаем выбранные признаки на предмет пропусков и выбросов:

### Признак surgery

#### Распределение значений:

Мода:

In [18]:
print("мода", horse_sample.surgery.mode())

мода 0    1.0
dtype: float64


Распределение значений в долях:

In [19]:
print(horse_sample.surgery.value_counts(normalize = True))

1.0    0.597222
2.0    0.402778
Name: surgery, dtype: float64


Распределение значений, частоты:

In [20]:
print(horse_sample.surgery.value_counts())

1.0    172
2.0    116
Name: surgery, dtype: int64


* Категориальный показатель: 1-операция, 2-нет.<br>
* Мода, наиболее часто встречающееся значение: 1, операция.<br>
* В 60% случаев была проведена операция.<br>

#### Пропуски:

Количество пропусков - один:

In [21]:
print(horse_sample['surgery'].isna().mean().round(4)*100)  
print(horse_sample['surgery'].isna().sum())

0.35000000000000003
1


Фильтруем пропуски - запись с индексом 132, pain = 4: 

In [22]:
horse_sample[horse_sample.surgery.isna()]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
132,NaN,1,534572,38.0,48.0,20.0,3.0,1.0,4.0,NaN,1.0,73.0


Заполнение пропусков модой surgery для конкретного значения pain:

In [23]:
horse_sample.surgery.fillna(horse_sample[horse_sample.pain == 4].surgery.mode()[0], inplace = True)
horse_sample.surgery.value_counts()

1.0    173
2.0    116
Name: surgery, dtype: int64

* По смыслу хотелось бы подставить моду по значениям pain, т.к. в среднем чем болезненнее, тем вероятнее операция:<br>
horse_sample.surgery.fillna(horse.groupby('pain').surgery.transform(lambda x: x.mode()[0]), inplace=True)<br>
* Но это сделать нельзя, если в группирующем столбце есть пропуски, а в pain около 50 пропусков.<br>
* В данном случае пропуск surgery только один, можно посмотреть, что в этом случае pain = 4, и " вручную" заменить модой surgery для pain = 4.

### Признак age

#### Распределение значений:

Мода:

In [24]:
horse_sample.age.mode()

0    1
dtype: int64

Распределение значений, частоты:

In [25]:
horse_sample.age.value_counts()

1    265
9     24
Name: age, dtype: int64

Распределение значений в долях:

In [26]:
horse_sample.age.value_counts(normalize = True)

1    0.916955
9    0.083045
Name: age, dtype: float64

* Категориальный показатель: 1 - adult, 9 - young (а не "2" как заявлено в описании).<br>
* Мода, наиболее часто встречающееся значение: 1, взрослые.<br>
* 92% случаев - взрослые лошади.<br>

#### Пропуски:

Количество и доля пропусков - 0 пропусков, 289 непустых значений:

In [27]:
print(horse_sample['age'].isna().mean().round(4) * 100) 
print(horse_sample['age'].count())

0.0
289


Пропусков нет.

### Признак hospital_number

Уникальных значений 284:

In [28]:
horse_sample['hospital_number'].nunique()

284

Категориальный.

Количество и доля пропусков - 0 пропусков, 289 непустых значений:

In [29]:
print(horse_sample['hospital_number'].isna().mean().round(4) * 100)
print(horse_sample['hospital_number'].count())

0.0
289


Пропусков нет.

### Признак outcome

#### Распределение значений:

Мода:

In [30]:
horse_sample.outcome.mode()

0    1.0
dtype: float64

Распределение значений в долях:

In [31]:
horse_sample.outcome.value_counts(normalize = True)

1.0    0.604167
2.0    0.246528
3.0    0.149306
Name: outcome, dtype: float64

Распределение значений, частоты:

In [32]:
horse_sample.outcome.value_counts()

1.0    174
2.0     71
3.0     43
Name: outcome, dtype: int64

* Категориальный: 1 = lived,2 = died,3 = was euthanized.<br>
* Мода, наиболее часто встречающееся значение: 1, выжившие лошади.<br>
* 60% выживших, 25% смертей и 15% усыплений.<br>

#### Пропуски:

Количество пропусков - 1 пропуск:

In [33]:
print(horse_sample['outcome'].isna().mean().round(4) * 100) 
print(horse_sample['outcome'].isna().sum())

0.35000000000000003
1


Фильтруем пропуски:

In [34]:
horse_sample[horse_sample.outcome.isna()] 

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
132,1.0,1,534572,38.0,48.0,20.0,3.0,1.0,4.0,NaN,1.0,73.0


Распределение значений outcome в разрезе surgery-pain:

In [35]:
horse_sample.groupby(['surgery','pain']).outcome.value_counts()

surgery  pain  outcome
1.0      1.0   1.0         8
         2.0   1.0        15
               2.0         9
               3.0         8
         3.0   1.0        32
               2.0         5
               3.0         1
         4.0   2.0        14
               1.0         8
               3.0         5
         5.0   2.0        15
               1.0         9
               3.0         6
2.0      1.0   1.0        26
               2.0         2
               3.0         1
         2.0   1.0        18
               3.0         4
               2.0         3
         3.0   1.0        21
               3.0         4
               2.0         3
         4.0   3.0         5
               1.0         2
               2.0         2
         5.0   2.0         6
               1.0         2
Name: outcome, dtype: int64

Заполнение пропусков модой outcome для конкретных значений pain и surgery:

In [36]:
(horse_sample.outcome.fillna(
    horse_sample[(horse_sample.pain == 4) & (horse_sample.surgery == 1) ].outcome.mode()[0], inplace=True))
horse_sample.outcome.value_counts()

1.0    174
2.0     72
3.0     43
Name: outcome, dtype: int64

* Тот же случай, где был пропуск по surgery (pain=4, surgery заполнили 1).<br>
* В группировке по [surgery, pain] кол-ва значений хватает.<br>
* Но так как в pain есть пропуски, которые будут заполняться на основе значений других столбцов,<br>
то для большей корректности заполним пропуск outcome не с помощью groupby(['surgery','pain'])-transform,<br>
а на основе моды внутри группы pain=4, surgery=1.

### Признак rectal_temperature

#### Распределение значений:

Мода:

In [37]:
horse_sample.rectal_temperature.mode()

0    38.0
dtype: float64

IQR, разброс:

In [38]:
horse_stat = horse_sample.describe()
print("IQR", round(horse_stat['rectal_temperature']['75%'] - horse_stat['rectal_temperature']['25%'], 4))
print("Разброс", round(horse_stat['rectal_temperature']['max'] - horse_stat['rectal_temperature']['min'], 4))

IQR 0.7
Разброс 5.4


Статистики:

In [39]:
rect_temp_pre = horse_stat['rectal_temperature']
rect_temp_pre

count    233.000000
mean      38.168240
std        0.739539
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64

* Мода, наиболее часто встречающееся значение: 38 градусов.<br>
* Среднее 38.16, медиана 38.2, по всем мерам центральной тенденции - около 38, чуть выше нормы 37.8.
* В среднем значения отклоняются от среднего всего на 0.7 градуса.<br>
* IQR 0.7 градуса, разброс 5.4 градуса.<br>

#### Выбросы:

Границы выбросов по методу IQR:

In [40]:
rectal_temperature_lb = (horse_stat['rectal_temperature']['25%'] - 
                         (1.5 * (horse_stat['rectal_temperature']['75%']-horse_stat['rectal_temperature']['25%']))) 
rectal_temperature_ub = (horse_stat['rectal_temperature']['75%'] + 
                         (1.5 * (horse_stat['rectal_temperature']['75%']-horse_stat['rectal_temperature']['25%'])))
round(rectal_temperature_lb, 2), round(rectal_temperature_ub, 2)

(36.75, 39.55)

Распределение значений:

In [41]:
horse_sample.rectal_temperature.value_counts().sort_index()

35.4     1
36.0     1
36.1     1
36.4     1
36.5     2
36.6     1
36.8     1
36.9     1
37.0     2
37.1     3
37.2     7
37.3     6
37.4     3
37.5    11
37.6     7
37.7     7
37.8    16
37.9     7
38.0    25
38.1    11
38.2    16
38.3    17
38.4    11
38.5    18
38.6    12
38.7     7
38.8     6
38.9     3
39.0     4
39.1     2
39.2     5
39.3     4
39.4     3
39.5     4
39.6     1
39.7     1
39.9     1
40.0     1
40.3     2
40.8     1
Name: rectal_temperature, dtype: int64

Статистики до и после исключения потенциальных выбросов:

In [42]:
horse_excl_out_rect_temp = (horse_sample[horse_sample.rectal_temperature.
                                         between(rectal_temperature_lb, rectal_temperature_ub, inclusive = True)])
print("before", rect_temp_pre)
print('')
print("after", horse_excl_out_rect_temp.describe()['rectal_temperature'])

before count    233.000000
mean      38.168240
std        0.739539
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64

after count    219.000000
mean      38.169406
std        0.575991
min       36.800000
25%       37.800000
50%       38.200000
75%       38.500000
max       39.500000
Name: rectal_temperature, dtype: float64


C:\Users\YULIAN~2\AppData\Local\Temp/ipykernel_10920/1133261043.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  horse_excl_out_rect_temp = (horse_sample[horse_sample.rectal_temperature.


Фильтруем потенциальные выбросы:

In [43]:
rect_temp_out = pd.concat([horse_sample, horse_excl_out_rect_temp]).drop_duplicates(keep = False)
rect_temp_out[rect_temp_out.rectal_temperature.notnull()]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
20,1.0,1,530157,39.9,72.0,60.0,1.0,4.0,5.0,1.0,NaN,6.1
44,1.0,1,535407,35.4,140.0,24.0,3.0,NaN,4.0,3.0,2.0,69.0
54,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,2.0,3.0,4.5,8.1
75,1.0,9,534092,39.7,100.0,NaN,3.0,NaN,2.0,3.0,2.0,57.0
80,1.0,1,527518,36.4,98.0,35.0,3.0,2.0,4.0,2.0,3.6,6.4
91,2.0,1,529461,40.3,114.0,36.0,3.0,3.0,2.0,2.0,4.5,8.1
99,2.0,1,530002,39.6,108.0,51.0,3.0,3.0,2.0,1.0,2.6,8.0
118,1.0,1,533983,36.5,78.0,30.0,1.0,1.0,5.0,1.0,1.0,75.0
141,2.0,1,522979,36.0,42.0,30.0,NaN,NaN,NaN,2.0,NaN,6.8
238,2.0,1,528702,36.1,88.0,NaN,3.0,2.0,3.0,3.0,4.8,7.0


* На взгляд и в левом и в правом хвостах значения увеличиваются постепенно, без скачков - на ошибки не похоже.<br>
* Меры центральной тенденции схожи, iqr и ср.кв.откл тоже - не похоже на наличие выбросов. Или выбросы и в правом и в левом конце балансируют друг друга.<br>

* Статистики почти не изменились при исключении выбросов.<br>
* Будем считать, что у значений в правом и левом концах естественные причины и их нужно оставить в выборке. Низкая температура может быть в случае шока, высокая в случае инфекции, а исключение выбросов по правилу iqr исключает/дискриминирует эти случаи.
* Для проверки ошибочности можно еще проверить выбросы на согласованность со значениями других признаков, например, temperature_of_extremities.

#### Пропуски:

Кол-во пропусков - 19% пропусков, 56 пропусков:

In [44]:
print(horse_sample['rectal_temperature'].isna().mean().round(4) * 100) 
print(horse_sample['rectal_temperature'].isna().sum()) 

19.38
56


Фильтруем пропуски:

In [45]:
horse_sample[horse_sample.rectal_temperature.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
5,2.0,1,528355,NaN,NaN,NaN,2.0,2.0,2.0,1.0,NaN,NaN
7,1.0,1,529607,NaN,60.0,NaN,3.0,2.0,NaN,2.0,NaN,8.3
8,2.0,1,530051,NaN,80.0,36.0,3.0,4.0,4.0,3.0,NaN,6.2
16,1.0,9,5301219,NaN,128.0,36.0,3.0,3.0,4.0,2.0,4.7,7.8
28,1.0,1,5279442,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


Медиана в разрезе temperature_of_extremities, незначительно отличаются между группами:

In [46]:
horse_sample.groupby('temperature_of_extremities').rectal_temperature.median()

temperature_of_extremities
1.0    38.20
2.0    38.20
3.0    38.10
4.0    38.45
Name: rectal_temperature, dtype: float64

Заполнение пропусков медианой rectal_temperature в разрезе значений temperature_of_extremities:

In [47]:
(horse_sample.rectal_temperature.fillna(horse_sample.groupby('temperature_of_extremities').
                                        rectal_temperature.transform('median'), inplace = True))

Медиана в разрезе temperature_of_extremities после заполнения:

In [48]:
horse_sample.groupby('temperature_of_extremities').rectal_temperature.median()

temperature_of_extremities
1.0    38.20
2.0    38.20
3.0    38.10
4.0    38.45
Name: rectal_temperature, dtype: float64

Фильтруем оставшиеся пропуски:

In [49]:
horse_sample[horse_sample.rectal_temperature.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
28,1.0,1,5279442,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
56,1.0,1,528872,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,6.7
74,1.0,9,5292929,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.9
87,2.0,1,528006,NaN,40.0,16.0,NaN,NaN,NaN,3.0,3.9,7.0
96,2.0,1,530319,NaN,84.0,36.0,NaN,1.0,NaN,1.0,NaN,8.5


Медиана в разрезе surgery-outcome:

In [50]:
print(horse_sample.groupby(['surgery', 'outcome']).rectal_temperature.median())

surgery  outcome
1.0      1.0        38.2
         2.0        38.1
         3.0        38.1
2.0      1.0        38.2
         2.0        38.1
         3.0        38.1
Name: rectal_temperature, dtype: float64


Заполняем пропуски медианой rectal_temperature в разрезе значений surgery-outcome. Смотрим медианы в разрезе surgery-outcome после заполнения:

In [51]:
(horse_sample.rectal_temperature.fillna(horse_sample.groupby(['surgery', 'outcome']).
                                        rectal_temperature.transform('median'), inplace = True))
horse_sample.groupby(['surgery', 'outcome']).rectal_temperature.median()

surgery  outcome
1.0      1.0        38.2
         2.0        38.1
         3.0        38.1
2.0      1.0        38.2
         2.0        38.1
         3.0        38.1
Name: rectal_temperature, dtype: float64

Статистики до и после заполнения:

In [52]:
print("before", rect_temp_pre)
print("after", horse_sample.describe()['rectal_temperature'])

before count    233.000000
mean      38.168240
std        0.739539
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64
after count    289.000000
mean      38.169377
std        0.665641
min       35.400000
25%       37.900000
50%       38.100000
75%       38.500000
max       40.800000
Name: rectal_temperature, dtype: float64


* rectal_temperature это числовой показатель, у которого есть смежный по смыслу temperature_of_extremities.
* Подставляем медиану на основе группировки по temperature_of_extremities.
* Но в temperature_of_extremities есть пропуски, для этих случаев подставляем медиану по surgery и outcome.
* Surgery и outcome единственные полностью (с учетом одного заполненного ранее пропуска) заполненные смысловые столбцы, значения которых наиболее вероятно связаны с другими показателями. Столбец type_of_lesion судя по описанию имеет слишком много значений, потеря обобщения. К тому же в нем есть нулевые значения, то есть пропуски. Значение столбца surgical_lesion не до конца ясно и кажется менее надежным для заполнения пропусков.
* После подстановки групповые медианы не изменились. Общие статистики изменились незначительно.

### Признак pulse

#### Распределение значений:

IQR, разброс:

In [53]:
horse_stat = horse_sample.describe()
print(horse_stat['pulse']['75%'] - horse_stat['pulse']['25%'])
print(horse_stat['pulse']['max'] - horse_stat['pulse']['min'])

40.0
154.0


Статистики:

In [54]:
puls_pre = horse_stat['pulse']
puls_pre 

count    267.000000
mean      71.842697
std       28.866519
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

* Среднее 71.8, медиана 64 уд/мин, среднее выше медианы, вероятно распределение скошено вправо. Норма для взрослых лошадей 30-40, то есть в среднем пульс повышен.<br>
* В среднем значения отклоняются от среднего на 28.9, учитывая норму - довольно значительная вариация.<br>
* IQR 40, разброс 154.<br>
* 75% случаев показатель меньше 88, норма 30-40, max 184 кажется запредельным, возможно выброс или ошибка.

#### Выбросы:

Границы выбросов по методу IQR. Отрицательное значение говорит о значительной вариации:

In [55]:
pulse_lb = horse_stat['pulse']['25%'] - (1.5 * (horse_stat['pulse']['75%'] - horse_stat['pulse']['25%'])) 
pulse_ub = horse_stat['pulse']['75%'] + (1.5 * (horse_stat['pulse']['75%'] - horse_stat['pulse']['25%']))
pulse_lb, pulse_ub

(-12.0, 148.0)

Распределение значений:

In [56]:
horse_sample.pulse.value_counts().sort_index()

30.0      2
36.0      3
38.0      1
40.0     18
42.0     10
44.0     11
45.0      2
46.0      1
48.0     26
49.0      1
50.0      6
52.0     11
54.0      7
56.0      7
60.0     25
64.0      8
65.0      1
66.0      6
68.0      5
70.0      3
72.0     10
75.0      1
76.0      4
78.0      3
80.0      7
82.0      1
84.0      8
86.0      3
88.0     12
90.0      3
92.0      4
96.0      7
98.0      1
100.0     9
104.0     5
108.0     3
110.0     1
112.0     3
114.0     2
120.0    10
124.0     2
128.0     1
129.0     1
130.0     2
132.0     1
136.0     1
140.0     2
146.0     1
150.0     2
160.0     1
164.0     1
184.0     1
Name: pulse, dtype: int64

Статистики до и после исключения потенциальных выбросов:

In [57]:
horse_excl_out_pulse = horse_sample[horse_sample.pulse.between(pulse_lb, pulse_ub, inclusive = True)]
print("before", puls_pre)
print('')
print("after", horse_excl_out_pulse.describe()['pulse'])

before count    267.000000
mean      71.842697
std       28.866519
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

after count    262.000000
mean      70.129771
std       26.248269
min       30.000000
25%       48.000000
50%       62.000000
75%       88.000000
max      146.000000
Name: pulse, dtype: float64


C:\Users\YULIAN~2\AppData\Local\Temp/ipykernel_10920/4272104808.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  horse_excl_out_pulse = horse_sample[horse_sample.pulse.between(pulse_lb, pulse_ub, inclusive = True)]


Фильтруем потенциальные выбросы:

In [58]:
pulse_out = pd.concat([horse_sample, horse_excl_out_pulse]).drop_duplicates(keep = False)
pulse_out[pulse_out.pulse.notnull()]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
3,1.0,9,5290409,39.1,164.0,84.0,4.0,4.0,2.0,2.0,5.3,7.2
41,2.0,9,5288249,39.0,150.0,72.0,NaN,NaN,NaN,1.0,0.1,8.5
55,1.0,9,5282839,38.6,160.0,20.0,3.0,4.0,3.0,2.0,NaN,NaN
255,1.0,9,5294539,38.8,184.0,84.0,1.0,3.0,4.0,2.0,NaN,3.3
275,1.0,9,5297159,38.8,150.0,50.0,1.0,2.0,5.0,2.0,NaN,6.2


Исключаем выброс:

In [59]:
horse_sample = horse_sample.drop(horse_sample[horse_sample.pulse == 184].index)

* В правом хвосте значения увеличиваются постепенно, без скачков до 164, потом скачок на 184 - может быть ошибка ввода/измерения.<br>
* Левый хвост начинается с нижней границы нормы - странно, но не похоже на ошибку. Возможно при таких болезнях этот показатель ниже нормы не бывает.<br>
<br>
* Исключение выбросов немного изменило среднее, ср.кв.откл и медиану.<br>
* Будем считать, что у выбросов естественные причины и их нужно оставить в выборке, кроме значения 184. В случае с значением 184 была операция, сильная боль, смерть - значения разных показателей выглядят вполне согласованно.<br>
* Можно сгруппировать случаи с сильной болью, операцией, смертью и посмотреть какой там интверал значений pulse. Можно попробовать выяснить, является ли такое значение теоретически возможным.<br>
* Но на данный момент, так как есть скачок в значениях - исключим 184.<br>

#### Пропуски:

Кол-во пропусков - 7% пропусков, 22 пропуска:

In [60]:
print(horse_sample['pulse'].isna().mean().round(4) * 100) 
print(horse_sample['pulse'].isna().sum())

7.64
22


Фильтруем пропуски:

In [61]:
horse_sample[horse_sample.pulse.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
5,2.0,1,528355,38.20,NaN,NaN,2.0,2.0,2.0,1.0,NaN,NaN
28,1.0,1,5279442,38.10,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
52,2.0,1,529483,38.20,NaN,NaN,1.0,1.0,1.0,1.0,NaN,7.7
56,1.0,1,528872,38.20,NaN,NaN,NaN,NaN,NaN,1.0,NaN,6.7
58,1.0,1,528298,38.45,NaN,20.0,4.0,3.0,5.0,2.0,NaN,5.9


Медиана в разрезе surgery-outcome:

In [62]:
print(horse_sample.groupby(['surgery', 'outcome']).pulse.median()) 

surgery  outcome
1.0      1.0        60.0
         2.0        84.0
         3.0        80.0
2.0      1.0        52.0
         2.0        90.0
         3.0        82.0
Name: pulse, dtype: float64


Заполнение пропусков медианой в разрезе значений surgery-outcome:

In [63]:
(horse_sample.pulse.fillna(horse_sample.groupby(['surgery', 'outcome']).
                                        pulse.transform('median'), inplace = True))

Медиана в разрезе surgery-outcome после заполнения пропусков:

In [64]:
horse_sample.groupby(['surgery', 'outcome']).pulse.median()

surgery  outcome
1.0      1.0        60.0
         2.0        84.0
         3.0        80.0
2.0      1.0        52.0
         2.0        90.0
         3.0        82.0
Name: pulse, dtype: float64

Статистики до и после заполнения:

In [65]:
print("before", puls_pre)
print('')
print("after", horse_sample.describe()['pulse'])

before count    267.000000
mean      71.842697
std       28.866519
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

after count    288.000000
mean      71.277778
std       27.289120
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      164.000000
Name: pulse, dtype: float64


* Числовой показатель, подставляем медиану на основе группировки по surgery и outcome.<br>
* После подстановки групповые медианы не изменились. Общие статистики изменились незначительно.

### Признак respiratory_rate

#### Распределение значений:

IQR, разброс:

In [66]:
horse_stat = horse_sample.describe()
print(horse_stat['respiratory_rate']['75%'] - horse_stat['respiratory_rate']['25%'])
print(horse_stat['respiratory_rate']['max'] - horse_stat['respiratory_rate']['min'])

18.0
88.0


Статистики:

In [67]:
resp_rat_pre = horse_stat['respiratory_rate']
resp_rat_pre

count    233.000000
mean      30.261803
std       17.574108
min        8.000000
25%       18.000000
50%       24.000000
75%       36.000000
max       96.000000
Name: respiratory_rate, dtype: float64

* Среднее 30.3, медиана 24, среднее выше медианы, вероятно распределение скошено вправо. Норма 8-10, то есть в среднем значение повышенное.<br>
* В среднем значения отклоняются от среднего на 17.6, учитывая норму - довольно значительная вариация.<br>
* IQR 18, разброс 88, учитывая норму  - очень большой разброс.<br>
* 75% случаев показатель меньше 36, max в 96 возможно выброс или ошибка.<br>
* Странно, что минимальное значение в выборке является нижней границей нормы - 8. Возможно при таких болезнях этот показатель ниже нормы не бывает.

#### Выбросы:

Границы выбросов методом IQR. Отрицательное значение говорит о значительной вариации:

In [68]:
respiratory_rate_lb = (horse_stat['respiratory_rate']['25%'] - 
                       (1.5 * (horse_stat['respiratory_rate']['75%'] - horse_stat['respiratory_rate']['25%']))) 
respiratory_rate_ub = (horse_stat['respiratory_rate']['75%'] + 
                       (1.5 * (horse_stat['respiratory_rate']['75%'] - horse_stat['respiratory_rate']['25%'])))
respiratory_rate_lb, respiratory_rate_ub 

(-9.0, 63.0)

Распределение значений:

In [69]:
horse_sample.respiratory_rate.value_counts().sort_index()

8.0      1
9.0      2
10.0     3
12.0    19
13.0     1
14.0     4
15.0     1
16.0    21
18.0     8
20.0    28
21.0     2
22.0     2
23.0     1
24.0    25
25.0     1
26.0     1
28.0    12
30.0    16
32.0    11
34.0     1
35.0     3
36.0    16
40.0    17
42.0     3
44.0     2
48.0     6
50.0     2
51.0     2
52.0     1
58.0     1
60.0     4
66.0     1
68.0     3
70.0     2
72.0     1
80.0     3
84.0     1
88.0     1
90.0     2
96.0     2
Name: respiratory_rate, dtype: int64

Статистики после исключения потенциальных выбросов:

In [70]:
horse_excl_out_resp_rat = (horse_sample[horse_sample.respiratory_rate.
                                        between(respiratory_rate_lb, respiratory_rate_ub, inclusive = True)])
print("before",resp_rat_pre)
print('')
print("after",horse_excl_out_resp_rat.describe()['respiratory_rate'])

before count    233.000000
mean      30.261803
std       17.574108
min        8.000000
25%       18.000000
50%       24.000000
75%       36.000000
max       96.000000
Name: respiratory_rate, dtype: float64

after count    217.000000
mean      26.658986
std       11.579589
min        8.000000
25%       18.000000
50%       24.000000
75%       35.000000
max       60.000000
Name: respiratory_rate, dtype: float64


C:\Users\YULIAN~2\AppData\Local\Temp/ipykernel_10920/345136059.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  horse_excl_out_resp_rat = (horse_sample[horse_sample.respiratory_rate.


Фильтруем потенциальные выбросы:

In [71]:
resp_rat_out = pd.concat([horse_sample, horse_excl_out_resp_rat]).drop_duplicates(keep = False)
resp_rat_out[resp_rat_out.respiratory_rate.notnull()]

,surgery,age,hospital_number,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
3,1.0,9,5290409,39.10,164.0,84.0,4.0,4.0,2.0,2.0,5.3,7.2
39,1.0,9,5277409,39.20,146.0,96.0,NaN,NaN,NaN,2.0,NaN,NaN
41,2.0,9,5288249,39.00,150.0,72.0,NaN,NaN,NaN,1.0,0.1,8.5
82,1.0,9,5290759,38.10,100.0,80.0,3.0,1.0,3.0,1.0,NaN,5.7
84,1.0,1,529849,37.80,60.0,80.0,1.0,3.0,2.0,1.0,NaN,4.5
103,1.0,9,5292489,38.00,140.0,68.0,1.0,2.0,3.0,1.0,NaN,5.3
106,1.0,1,528668,38.30,52.0,96.0,NaN,NaN,NaN,1.0,NaN,6.1
120,1.0,1,528682,39.40,54.0,66.0,1.0,2.0,2.0,1.0,NaN,6.0
125,1.0,1,528461,38.00,42.0,68.0,4.0,2.0,3.0,1.0,NaN,7.6
186,1.0,1,5278331,39.30,64.0,90.0,2.0,1.0,NaN,1.0,NaN,6.7


Исключаем признак из данных:

In [72]:
horse_sample = horse_sample.drop(['respiratory_rate'], axis = 1)

* В правом хвосте значения увеличиваются постепенно, без скачков - не похоже на ошибку.<br>
* В рез-те искл. выбросов среднее несколько снизилось, приблизилось к медиане.<br>
* Будем считать, что у значений в правом конце естественные причины.
* В описании говорится о значительных колебаниях этого показателя и сомнениях в его полезности. Исключим этот признак из анализа.

### Признак temperature_of_extremities

#### Распределение значений:

Мода:

In [73]:
horse_sample.temperature_of_extremities.mode()

0    3.0
dtype: float64

Распределение значений:

In [74]:
horse_sample.temperature_of_extremities.value_counts()

3.0    102
1.0     75
2.0     30
4.0     25
Name: temperature_of_extremities, dtype: int64

Распределение значений в долях:

In [75]:
horse_sample.temperature_of_extremities.value_counts(normalize = True)

3.0    0.439655
1.0    0.323276
2.0    0.129310
4.0    0.107759
Name: temperature_of_extremities, dtype: float64

* Категориальный  1 Normal, 2 Warm, 3 Cool, 4 Cold.<br>
* Мода, наиболее часто встречающееся значение: 3 , то есть не норма, возможный шок.<br>
* 44% случаев - возможный шок, при этом треть - норма.

#### Пропуски:

Кол-вo пропусков - 19% пропусков, 56 пропусков:

In [76]:
print(horse_sample['temperature_of_extremities'].isna().mean().round(4) * 100)
print(horse_sample['temperature_of_extremities'].isna().sum())

19.439999999999998
56


Исключаем признак из данных:

In [77]:
horse_sample.drop(['temperature_of_extremities'], axis = 1, inplace = True)

Смежные по смыслу столбцы с rectal_temperature, поэтому можно использовать
temperature_of_extremities для заполнения пропусков rectal_temperature,
а сам temperature_of_extremities не использовать.

### Признак abdominal_distension

#### Распределение значений:

Мода:

In [78]:
horse_sample.abdominal_distension.mode()

0    1.0
dtype: float64

Распределение значений в долях:

In [79]:
horse_sample.abdominal_distension.value_counts(normalize = True)

1.0    0.314655
2.0    0.275862
3.0    0.262931
4.0    0.146552
Name: abdominal_distension, dtype: float64

Распределение значений:

In [80]:
horse_sample.abdominal_distension.value_counts()

1.0    73
2.0    64
3.0    61
4.0    34
Name: abdominal_distension, dtype: int64

* Категориальный: 1 none, 2 slight, 3 moderate, 4 severe.<br>
* Мода, наиболее часто встречающееся значение: 1 , то есть норма, скорее всего без боли, без необходимости операции.<br>
* Относительно равномерное распределение между значениями, норма - 31 %, 15% - тяжелая форма с вероятной операцией.

#### Пропуски:

Кол-во пропусков - 19% пропусков,  56 пропусков:

In [81]:
print(horse_sample['abdominal_distension'].isna().mean().round(4) * 100) 
print(horse_sample['abdominal_distension'].isna().sum())

19.439999999999998
56


Фильтруем пропуски:

In [82]:
horse_sample[horse_sample.abdominal_distension.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
4,2.0,1,530255,37.3,104.0,NaN,NaN,2.0,NaN,7.4
17,2.0,1,526639,37.5,48.0,NaN,NaN,1.0,NaN,NaN
23,1.0,9,534998,38.3,130.0,NaN,2.0,1.0,NaN,70.0
25,2.0,1,529518,37.8,60.0,NaN,NaN,1.0,NaN,NaN
28,1.0,1,5279442,38.1,84.0,NaN,NaN,2.0,NaN,NaN


Заполняем пропуски модой в разрезе значений surgery-outcome:

In [83]:
(horse_sample.abdominal_distension.fillna(horse_sample.groupby(['surgery','outcome']).
                                          abdominal_distension.transform(lambda x: x.mode()[0]), inplace = True))
horse_sample.abdominal_distension.value_counts()

1.0    104
3.0     74
2.0     64
4.0     46
Name: abdominal_distension, dtype: int64

### Признак pain

#### Распределение значений:

Мода:

In [84]:
horse_sample.pain.mode()

0    3.0
dtype: float64

Распределение значений в долях:

In [85]:
horse_sample.pain.value_counts(normalize = True)

3.0    0.282051
2.0    0.243590
5.0    0.162393
1.0    0.158120
4.0    0.153846
Name: pain, dtype: float64

Распределение значений:

In [86]:
horse_sample.pain.value_counts()

3.0    66
2.0    57
5.0    38
1.0    37
4.0    36
Name: pain, dtype: int64

* Категориальный: 1 alert, no pain, 2 = depressed,3 = intermittent mild pain,4 = intermittent severe pain,5 = continuous severe pain.<br>
* Мода, наиболее часто встречающееся значение: 3, прерывистая неострая боль.<br>
* Относительно равномерное распределение между значениями, мода - 28 %.<br>

#### Пропуски:

Кол-во пропусков - 18% пропусков, 54 пропусков:

In [87]:
print(horse_sample['pain'].isna().mean().round(4) * 100) 
print(horse_sample['pain'].isna().sum())

18.75
54


Фильтруем пропуски:

In [88]:
horse_sample[horse_sample.pain.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
4,2.0,1,530255,37.3,104.0,4.0,NaN,2.0,NaN,7.4
7,1.0,1,529607,38.1,60.0,2.0,NaN,2.0,NaN,8.3
17,2.0,1,526639,37.5,48.0,1.0,NaN,1.0,NaN,NaN
19,2.0,1,532110,39.4,110.0,3.0,NaN,1.0,NaN,8.7
24,1.0,1,533692,38.1,60.0,3.0,NaN,1.0,NaN,65.0


Заполнение пропусков модой в разрезе значений surgery-outcome:

In [89]:
(horse_sample.pain.fillna(horse_sample.groupby(['surgery','outcome']).
                          pain.transform(lambda x: x.value_counts().idxmax()), inplace = True))
horse_sample.pain.value_counts()

3.0    86
2.0    64
5.0    50
1.0    50
4.0    38
Name: pain, dtype: int64

### Признак abdomcentesis total protein

#### Распределение значений:

IQR, разброс:

In [90]:
horse_stat = horse_sample.describe()
print(horse_stat['abdomcentesis_total_protein']['75%'] - horse_stat['abdomcentesis_total_protein']['25%'])
print(horse_stat['abdomcentesis_total_protein']['max'] - horse_stat['abdomcentesis_total_protein']['min'])

1.9
10.0


Статистики:

In [91]:
abd_tot_prot_pre = horse_stat['abdomcentesis_total_protein']
abd_tot_prot_pre

count    99.000000
mean      2.974747
std       1.954604
min       0.100000
25%       2.000000
50%       2.100000
75%       3.900000
max      10.100000
Name: abdomcentesis_total_protein, dtype: float64

* Среднее 3, медиана 2.1, среднее выше медианы, вероятно распределение немного скошено вправо. В среднем значения отклоняются от среднего на 1.9.<br>
* IQR 1.9, разброс 10.<br>
* 75% случаев показатель меньше 3.9, max в 10 возможно выброс или ошибка.<br>

#### Выбросы:

Границы потенциальных выбросов по методу IQR. Отрицательное значение говорит о значительной вариации:

In [92]:
abdomcentesis_total_protein_lb = (horse_stat['abdomcentesis_total_protein']['25%'] - 
                                  (1.5 * (horse_stat['abdomcentesis_total_protein']['75%'] - 
                                          horse_stat['abdomcentesis_total_protein']['25%']))) 
abdomcentesis_total_protein_ub = (horse_stat['abdomcentesis_total_protein']['75%'] + 
                                  (1.5 * (horse_stat['abdomcentesis_total_protein']['75%'] - 
                                          horse_stat['abdomcentesis_total_protein']['25%'])))
round(abdomcentesis_total_protein_lb, 2), round(abdomcentesis_total_protein_ub, 2)

(-0.85, 6.75)

Распределение значений:

In [93]:
horse_sample.abdomcentesis_total_protein.value_counts().sort_index()

0.1      1
1.0     17
1.3      1
1.4      1
1.5      1
1.6      2
2.0     26
2.1      1
2.2      1
2.3      2
2.5      1
2.6      3
2.8      3
2.9      1
3.0      2
3.2      1
3.3      1
3.4      3
3.6      3
3.7      1
3.9      3
4.1      2
4.3      2
4.4      1
4.5      2
4.7      1
4.8      1
5.0      3
5.2      1
5.3      2
6.0      2
6.6      1
7.0      2
7.4      1
8.0      1
10.0     1
10.1     1
Name: abdomcentesis_total_protein, dtype: int64

Статистики после исключения выбросов:

In [94]:
horse_excl_out_abd_prot = (horse_sample[horse_sample.abdomcentesis_total_protein.
                                        between(1, abdomcentesis_total_protein_ub, inclusive = True)])
# abdomcentesis_total_protein_lb или 1 (следующее после подозрительного 0.1) - разницы в статистиках почти нет.
print("before",abd_tot_prot_pre)
print('')
print("after",horse_excl_out_abd_prot.describe()['abdomcentesis_total_protein'])

before count    99.000000
mean      2.974747
std       1.954604
min       0.100000
25%       2.000000
50%       2.100000
75%       3.900000
max      10.100000
Name: abdomcentesis_total_protein, dtype: float64

after count    92.000000
mean      2.661957
std       1.405550
min       1.000000
25%       2.000000
50%       2.000000
75%       3.600000
max       6.600000
Name: abdomcentesis_total_protein, dtype: float64


C:\Users\YULIAN~2\AppData\Local\Temp/ipykernel_10920/885684618.py:1: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  horse_excl_out_abd_prot = (horse_sample[horse_sample.abdomcentesis_total_protein.


Фильтруем потенциальные выбросы:

In [95]:
abd_prot_out = pd.concat([horse_sample, horse_excl_out_abd_prot]).drop_duplicates(keep = False)
abd_prot_out[abd_prot_out.abdomcentesis_total_protein.notnull()]

,surgery,age,hospital_number,rectal_temperature,pulse,abdominal_distension,pain,outcome,abdomcentesis_total_protein,total_protein
31,2.0,1,530242,37.2,108.0,2.0,2.0,3.0,7.4,8.2
41,2.0,9,5288249,39.0,150.0,1.0,1.0,1.0,0.1,8.5
71,1.0,1,529424,37.8,48.0,1.0,1.0,1.0,7.0,5.9
113,1.0,1,527933,36.8,60.0,3.0,5.0,2.0,10.0,NaN
211,1.0,1,529893,38.0,86.0,4.0,2.0,2.0,10.1,5.5
225,2.0,1,528931,38.4,54.0,1.0,1.0,1.0,8.0,7.2
284,2.0,1,530439,38.5,54.0,1.0,3.0,1.0,7.0,6.8


* В правом хвосте значения увеличиваются постепенно до 8, потом 10 и 10.1 - может ошибка ввода/измерения.<br>
* В левом хвосте значения увеличиваются постепенно, кроме 0.1 - похоже на ошибку.<br>
* В рез-те исключения выбросов среднее несколько снизилось, приблизилось к медиане.<br>
* Считаю, что у выбросов естественные причины, кроме 0.1, 10 и 10.1 - их лучше исключить как потенциальные ошибки. Для проверки ошибочности можно посмотреть согласованность со значениями других показателей.<br>
* Так как по этому признаку много пропусков, то исключим его.

#### Пропуски:

Кол-во пропусков - 66% пропусков:

In [96]:
horse_sample['abdomcentesis_total_protein'].isna().mean().round(4) * 100

65.62

Исключаем признак из данных:

In [97]:
horse_sample.drop(['abdomcentesis_total_protein'], axis = 1, inplace = True)

* 66% пропусков, больше половины!<br>
* Не брать в рассмотрение этот показатель, вместо него выбрать какой-то смежный, может быть peristalsis, который тоже отражает состояние кишечника.

### Признак total_protein

#### Распределение значений:

IQR, разброс:

In [98]:
horse_stat = horse_sample.describe()
print(horse_stat['total_protein']['75%'] - horse_stat['total_protein']['25%'])
print(horse_stat['total_protein']['max'] - horse_stat['total_protein']['min'])

51.5
85.0


Мода:

In [99]:
horse_sample.total_protein.mode() 

0    6.5
1    7.0
dtype: float64

Статистики:

In [100]:
tot_prot_pre = horse_stat['total_protein']
tot_prot_pre

count    257.000000
mean      25.149416
std       27.774725
min        4.000000
25%        6.500000
50%        7.500000
75%       58.000000
max       89.000000
Name: total_protein, dtype: float64

* Среднее 25, медиана 7.5, среднее значительно выше медианы, вероятно распределение значительно скошено вправо/есть выбросы. По медиане - в среднем в норме.<br>
* Бимодальное распределение 6.5 и 7 - значения нормы.<br>
* В среднем значения отклоняются от среднего на 27.8, учитывая норму - вариация значительная.<br>
* IQR 51.5, разброс 85. Для нормы в 6-7 показатели разброса очень велики, вероятно есть выбросы в правом конце распределения.
* max в 89 возможно выброс или ошибка.

#### Выбросы:

Границы потенциальных выбросов по методу IQR. Обе границы вышли за min, max - очень значительная вариация:

In [101]:
total_protein_lb = (horse_stat['total_protein']['25%'] - 
                    (1.5 * (horse_stat['total_protein']['75%'] - horse_stat['total_protein']['25%']))) 
total_protein_ub = (horse_stat['total_protein']['75%'] + 
                    (1.5 * (horse_stat['total_protein']['75%'] - horse_stat['total_protein']['25%'])))
total_protein_lb, total_protein_ub

(-70.75, 135.25)

Распределение значений меньше 13:

In [102]:
horse_sample.total_protein[horse_sample.total_protein <= 13].value_counts().sort_index()

4.0      1
4.5      2
4.6      1
4.7      1
4.9      2
5.0      1
5.3      2
5.5      4
5.7      3
5.8      1
5.9      6
6.0      7
6.1      5
6.2      6
6.3      5
6.4      4
6.5     14
6.6     10
6.7      8
6.8      6
6.9      2
7.0     14
7.1      1
7.2      9
7.3      3
7.4      4
7.5     11
7.6      5
7.7      5
7.8      3
7.9      1
8.0      5
8.1      4
8.2      1
8.3      2
8.4      2
8.5      5
8.6      3
8.7      2
8.8      1
8.9      1
9.0      1
9.1      1
10.2     1
11.0     1
13.0     1
Name: total_protein, dtype: int64

Распределение значений больше 13:

In [103]:
horse_sample.total_protein[horse_sample.total_protein > 13].value_counts().sort_index()

46.0    1
50.0    1
51.0    1
53.0    1
54.0    2
55.0    2
56.0    3
57.0    2
58.0    4
59.0    1
60.0    3
61.0    3
62.0    3
63.0    1
64.0    5
65.0    8
66.0    4
67.0    4
68.0    2
69.0    3
70.0    5
71.0    2
72.0    1
73.0    1
74.0    2
75.0    3
76.0    1
77.0    2
80.0    2
81.0    2
82.0    1
85.0    1
86.0    1
89.0    1
Name: total_protein, dtype: int64

* В значениях есть скачок, странно делятся на две части: до 13 и после 46.<br>
* В обеих частях значения увеличиваются постепенно, без скачков - не похоже на ошибку.<br>
* Нужно предметное понимание возможных причин для таких подгрупп значений. Возможно, две части значений стоит рассмотреть отдельно.<br>
* Пока считаем, что у значений естественные причины.<br>

#### Пропуски:

Кол-во пропусков - 11% пропусков, 31 пропуск:

In [104]:
print(horse_sample['total_protein'].isna().mean().round(4) * 100)
print(horse_sample['total_protein'].isna().sum()) 

10.76
31


Фильтруем пропуски:

In [105]:
horse_sample[horse_sample.total_protein.isna()].head()

,surgery,age,hospital_number,rectal_temperature,pulse,abdominal_distension,pain,outcome,total_protein
5,2.0,1,528355,38.2,52.0,2.0,2.0,1.0,NaN
17,2.0,1,526639,37.5,48.0,1.0,1.0,1.0,NaN
25,2.0,1,529518,37.8,60.0,1.0,1.0,1.0,NaN
28,1.0,1,5279442,38.1,84.0,3.0,5.0,2.0,NaN
39,1.0,9,5277409,39.2,146.0,3.0,5.0,2.0,NaN


Медиана в разрезе surgery-outcome. Медиана для операции-эвтаназии очень высокая:

In [106]:
print(horse_sample.groupby(['surgery', 'outcome']).total_protein.median())

surgery  outcome
1.0      1.0         7.55
         2.0         7.10
         3.0        64.00
2.0      1.0         7.50
         2.0         6.50
         3.0         7.00
Name: total_protein, dtype: float64


Заполнение пропусков медианой в разрезе значений surgery-outcome:

In [107]:
(horse_sample.total_protein.fillna(horse_sample.groupby(['surgery', 'outcome']).
                                        total_protein.transform('median'), inplace = True))
horse_sample[horse_sample.total_protein.isna()] 

,surgery,age,hospital_number,rectal_temperature,pulse,abdominal_distension,pain,outcome,total_protein


Медиана в разрезе surgery-outcome после заполнения:

In [108]:
horse_sample.groupby(['surgery', 'outcome']).total_protein.median()

surgery  outcome
1.0      1.0         7.55
         2.0         7.10
         3.0        64.00
2.0      1.0         7.50
         2.0         6.50
         3.0         7.00
Name: total_protein, dtype: float64

Статистики до и после заполнения:

In [109]:
print("before", tot_prot_pre)
print("")
print("after", horse_sample.describe()['total_protein'])

before count    257.000000
mean      25.149416
std       27.774725
min        4.000000
25%        6.500000
50%        7.500000
75%       58.000000
max       89.000000
Name: total_protein, dtype: float64

after count    288.000000
mean      24.209028
std       27.249106
min        4.000000
25%        6.600000
50%        7.500000
75%       57.000000
max       89.000000
Name: total_protein, dtype: float64


* Числовой показатель, подставляем медиану на основе группировки по surgery и outcome.<br>
* После подстановки групповые медианы не изменились. Общие статистики изменились незначительно.

### Итоговый датафрейм

In [110]:
horse_sample.head()
horse_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   surgery               288 non-null    float64
 1   age                   288 non-null    int64  
 2   hospital_number       288 non-null    int64  
 3   rectal_temperature    288 non-null    float64
 4   pulse                 288 non-null    float64
 5   abdominal_distension  288 non-null    float64
 6   pain                  288 non-null    float64
 7   outcome               288 non-null    float64
 8   total_protein         288 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 22.5 KB


<a href="#0"><h4>Наверх</h4></a>